<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/boruta_suc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit
!pip install dcekit
!pip install boruta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 1.8 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.figure as figure
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from dcekit.variable_selection import search_high_rate_of_same_values, search_highly_correlated_variables
from boruta import BorutaPy



In [3]:
#データの読み込み
suppl = Chem.SDMolSupplier('logSdataset1290_2d.sdf')
mols = [mol for mol in suppl if mol is not None]

#RDkit記述子の計算
descriptor_names = [descriptor_name[0] for descriptor_name in Chem.Descriptors._descList]
descriptor_calculation = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
RDkit = [descriptor_calculation.CalcDescriptors(mol) for mol in mols]
df = pd.DataFrame(RDkit, columns = descriptor_names)
df.drop('MolLogP',axis=1,inplace=True)

#溶解度(logS)の値を取得
df['logS'] =  [mol.GetProp('logS') if 'logS' in mol.GetPropNames() else 'None' for mol in mols]

#ここから変数選択
df = df.astype(float)

#欠損のある変数を削除
df.dropna(axis=1,inplace=True)

#分散が０の変数削除
del_num1 = np.where(df.var() == 0)
df = df.drop(df.columns[del_num1], axis=1)

#変数選択（互いに相関関係にある変数の内一方を削除）
threshold_of_r = 0.95 #変数選択するときの相関係数の絶対値の閾値
corr_var = search_highly_correlated_variables(df, threshold_of_r)
df.drop(df.columns[corr_var], axis=1, inplace=True)

#同じ値を多くもつ変数の削除
inner_fold_number = 2 #CVでの分割数（予定）
rate_of_same_value = []
threshold_of_rate_of_same_value = (inner_fold_number-1)/inner_fold_number
for col in df.columns:
    same_value_number = df[col].value_counts()
    rate_of_same_value.append(float(same_value_number[same_value_number.index[0]] / df.shape[0]))
del_var_num = np.where(np.array(rate_of_same_value) >= threshold_of_rate_of_same_value)
df.drop(df.columns[del_var_num], axis=1, inplace=True)
print(df.shape)
# => (1290, 64)


[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerator
[13:10:18] DEPRECATION WARNING: please use MorganGenerat

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
(1290, 66)


In [4]:
#訓練・テストデータの分割と標準化
df_X = df.drop(['logS'],axis=1)
df_y = df.loc[:,['logS']]
X = df_X.values
y = df_y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

#StandardScalerを使って標準化
scaler_X = StandardScaler()
scaler_y = StandardScaler()
scaled_X_train = scaler_X.fit_transform(X_train)
scaled_y_train = scaler_y.fit_transform(y_train)
scaled_X_test = scaler_X.transform(X_test)
scaled_y_test = scaler_y.transform(y_test)

#Borutaによる変数選択
#pパーセンタイルの最適化（金子研の方法）
corr_list = []
for n in range(10000):
    shadow_features = np.random.rand(scaled_X_train.shape[0]).T
    corr = np.corrcoef(scaled_X_train, shadow_features,rowvar=False)[-1]
    corr = abs(corr[corr < 0.95])
    corr_list.append(corr.max())

corr_array = np.array(corr_list)
perc = 100 * (1-corr_array.max())

#Borutaの実施
feat_selector = BorutaPy(rf(),
n_estimators='auto',  # 特徴量の数に比例して、木の本数を増やす
verbose=0,
alpha=0.05, # 有意水準
max_iter=100, # 試行回数
perc = perc, #ランダム生成変数の重要度の何％を基準とするか
random_state=0
)
feat_selector.fit(scaled_X_train, scaled_y_train)
scaled_X_train_br = scaled_X_train[:,feat_selector.support_]
scaled_X_test_br = scaled_X_test[:,feat_selector.support_]
print('pパーセンタイル:',round(perc,2))
print('boruta後の変数の数:',scaled_X_train_br.shape[1])
#"""output
#pパーセンタイル:80.79
#boruta後の変数の数: 43
#"""


pパーセンタイル: 80.26
boruta後の変数の数: 46


In [ ]:
#scaled_pred_y_br.shape

(645,)

In [9]:
model = rf()
model_br = rf()
model.fit(scaled_X_train, scaled_y_train)
model_br.fit(scaled_X_train_br, scaled_y_train)
#
scaled_pred_y_br = model_br.predict(scaled_X_test_br) # Remove the flattening operation
pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br.reshape(-1, 1)) # Reshape the array to 2D
#scaled_pred_y =  model_br.predict((scaled_X_test))
#pred_y = scaler_y.inverse_transform(scaled_pred_y_.reshape(-1, 1))
#

#scaled_pred_y_br = np.ndarray.flatten(model_br.predict(scaled_X_test_br))
#pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)
scaled_pred_y = np.ndarray.flatten(model.predict(scaled_X_test))
pred_y = scaler_y.inverse_transform(scaled_pred_y.reshape(-1, 1))
#pred_y = scaler_y.inverse_transform(scaled_pred_y)

#
#print('r2_br:{}'.format(round(r2_score(y_test, pred_y_br),3))) # Use the correct variable name
#print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_br)),3))) # Use the correct variable name
#

print('r2_br:{}'.format(round(r2_score(y_test, pred_y_br),3)))
print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_br)),3)))
print('r2:{}'.format(round(r2_score(y_test, pred_y),3)))
print('RMSE:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y)),3)))
#"""output
#r2_br:0.883
#RMSE_br:0.697
#r2:0.883
#RMSE:0.699


r2_br:0.885
RMSE_br:0.693
r2:0.883
RMSE:0.7


In [ ]:
#もともとのコードのテストはなんだったのだろうか、、、、

In [8]:
#前回改善された部分
#model_br.fit(scaled_X_train_br, scaled_y_train)
#scaled_pred_y_br = np.ndarray.flatten(model_br.predict(scaled_X_test_br)) # This line flattens the array to 1D
#pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)

model_br.fit(scaled_X_train_br, scaled_y_train)
scaled_pred_y_br = model_br.predict(scaled_X_test_br) # Remove the flattening operation
pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br.reshape(-1, 1)) # Reshape the array to 2D
#scaled_pred_y = np.ndarray.flatten(model.predict(scaled_X_test))
#pred_y = scaler_y.inverse_transform(scaled_pred_y)

#print('r2_br:{}'.format(round(r2_score(y_test, pred_y_test_br),3))) # pred_y_test_br is not defined
#print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test_br)),3))) # pred_y_test_br is not defined
#print('r2:{}'.format(round(r2_score(y_test, pred_y_test),3))) # pred_y_test is not defined
#print('RMSE:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test)),3))) # pred_y_test is not defined

print('r2_br:{}'.format(round(r2_score(y_test, pred_y_br),3))) # Use the correct variable name
print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_br)),3))) # Use the correct variable name

r2_br:0.882
RMSE_br:0.702
